In [1]:
%load_ext autoreload
%autoreload 2

from model_utils import train_model, split_data, split_branches, get_model, set_pretrained_model_dropout, get_dataset, ModelSeeder
from config import Config
import json
import matplotlib.pyplot as plt
%matplotlib inline
import math
import random
import time
import onnx
import logging
from onnx_model_manager import OnnxModelManager
from onnxruntime.quantization import quantize_dynamic, QuantType
import os
import datasets
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from model_manager import ModelManager

In [2]:
saved_model_path = os.path.join("models", "awsw_main")
tokenizer = AutoTokenizer.from_pretrained('EleutherAI/gpt-neo-125M')
main_model = AutoModelForCausalLM.from_pretrained(saved_model_path)
base_model, _ = get_model("EleutherAI/gpt-neo-125M")
target_model = AutoModelForCausalLM.from_pretrained(saved_model_path)
device = torch.device('cpu')
model_manager = ModelManager(model=target_model, tokenizer=tokenizer, device=device)
print(f"Loaded base and main model to CPU")

Loaded base and main model to CPU


In [9]:
prompts = [
    ('<p><msg>c "Hey Remy!"<d><scn>park2<msg>Ry "Hey!"', "How are you?"),
    ('<p><msg>c "I was with Lorem today."<d><scn>park2<msg>Ad "Very nice."', "What do you think of Lorem?"),
    ('<p><msg>m "In Tatsu park, Adine and I sat down."', "Oh my god, Adine. What is this?"),
    ('<p><msg>m "I sat down on a chair in Anna\'s lab."', "What will we do here?"),
]

def sample_test(model_manager):
    for (past, prompt) in prompts:
        print(f"Prompt: {prompt}")
        reply = model_manager.say(past, prompt)
        print(f"Reply: {reply}")
        reply = model_manager.say(past, prompt, top_k = 50, top_p = 0.7)
        print(f"Reply [sampled]: {reply}")
        print("-" * 10)
sample_test(model_manager)

Prompt: How are you?
Reply: park2<msg>Ry "I'm fine."<p><msg>c "I'm fine."<p><msg>c "I'm fine."<p><msg>c "I'm fine."<p><msg>c "I'm fine."<p><msg>c "I'm fine."<p><msg>c "I'm fine."<p><msg>c "I'm
Reply [sampled]: park2<msg>Ry "Hey, I'm just about done here, but I think I've had to do something a little different."<p><msg>c "Yeah, I'm fine."<d><scn>park2<msg>Ry "I'm just having a little fun, I guess."<|endoftext|>
----------
Prompt: What do you think of Lorem?
Reply: park2<msg>Ad "I think he's a good student."<p><msg>c "I'm not sure, but I think he's a good student."<p><msg>c "I'm not sure, but I think I can handle it."<p><msg>c "I'm not sure, but I think I can handle it."<
Reply [sampled]: park2<msg>Ad "I'm not sure, but I think he's a good art critic."<p><msg>c "I can see that."<p><msg>c "I'm not sure what you're talking about."<d><scn>park2<msg>Ad "I think I should get going."<p><msg>c
----------
Prompt: Oh my god, Adine. What is this?
Reply: park<msg>Adine, I don't know. I don't want t

In [26]:
with torch.no_grad():
    for p1, p2, p3 in zip(main_model.parameters(), base_model.parameters(), target_model.parameters()):
        p3.data = torch.lerp(p1.data, p2.data, 0.5)

In [27]:
test_rps = [
    "Visit Lorem",
    "Meet with Lorem",
    "Visit Adine",
    "Fight Maverick",
    "Fight Adine",
    "Attack Adine"
]
for rp in test_rps:
    print(f'[Pytorch] {rp} -> {model_manager.say("", rp, top_k = 50, top_p = 0.7)}')
    print("-" * 10)
    
sample_test(model_manager)

[Pytorch] Visit Lorem -> loremapt<msg>Lo "Hey [player_name]!"<|endoftext|>
----------
[Pytorch] Meet with Lorem -> loremapt<msg>Lo "Hey [player_name]! How are you?"<|endoftext|>
----------
[Pytorch] Visit Adine -> adineapt<msg>Ad "Hey, [player_name]!"<|endoftext|>
----------
[Pytorch] Fight Maverick -> o2<msg>m "Maverick dodges my attack and comes out with a nice, wide, and cheap weapon. I think he's getting desperate."<d><scn>o2<msg>m "I'm just saying Maverick's actions are very similar to what I just said."<d><scn>o2<msg>m "I'm not sure if there is anything I can do to help, but I'll be right back."<d><scn>o2<msg>m "
----------
[Pytorch] Fight Adine -> cafe<msg>m "Adine barely avoids my attack, and I quickly dove behind her, trying to stay in cover as I dove into the water."<d><scn>cafe<msg>m "I quickly dove behind her, but she managed to get up and quickly punch me in the face, a fish in the line of fire."<d><scn>cafe<msg>m "I was just about to start on my second when I suddenly hea